In [127]:
# Packages used in this notebook
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# setting the starting url 
url_overview_köln = "https://www.wg-gesucht.de/wg-zimmer-in-Koeln.73.0.0.1.html"

In [98]:
# Testing 
# Sends a GET request to the URL to retrieve its content; The Website tryes to redicrect the request to a captcha page, reloading solves the problem
response = requests.get(url_overview_köln, allow_redirects=False)
# assert response.status_code == 200
print(response.status_code)
# Parse results
soup = BeautifulSoup(response.text, 'html.parser')

target_class = "ang_spalte_icons row_click"
elements = soup.find_all('td', class_=target_class)
links=[]
for element in elements:
    links.append(element.find('a').get('href'))
# Extracting all shared apartments
#for link in soup.find_all('a'):
#    print(link.get('href'))
print(links)
print(len(links))


302
[]
0


302 => Captcha -->nochmal probieren
200 => Okay

wnen die Klasse auf einer Seite nicht mehr gefunden werden kann, enthält diese keine Angebote mehr

In [148]:
page = 0
failed = 0
base_links=[]
campaign_links=[]

# main class for the links
target_class = "truncate_title noprint"

# class for the campaign links
campaign_class = "campaign_click"

last_page = page
cum_fails = 0
while page <= 20:
    # setting the url 
    url_overview_köln = "https://www.wg-gesucht.de/wg-zimmer-in-Koeln.73.0.1."+str(page)+".html"

    # Sends a GET request to the URL to retrieve its content; sometimes the Website tryes to redicrect the request to a captcha page, reloading solves the problem
    response = requests.get(url_overview_köln, allow_redirects=False)
    
    # test response code
    if response.status_code == 302:
        # try again
        failed += 1 
        cum_fails += 1
        continue 
    
    page += 1
    failed = 0

    # Parse results
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extracting all shared apartments
    elements = soup.find_all('h3', class_ = target_class)

    if len(elements) == 0:
        print("No more elements found")
        break

    
    for element in elements:
        for appartment in element.find_all('a', class_ = campaign_class):
            campaign_links.append(element.find('a').get('href'))
        for appartment in element.find_all('a', class_=''):
            base_links.append(element.find('a').get('href'))

    # just for debugging
    last_page = page -1

print("Base links: ")
print(base_links)

set_base_links = set(base_links)
base_links = list(set_base_links)
print("Anzahl der Base links:" + str(len(base_links)))



print("Campaign links: " )
print(campaign_links)

set_campaign_links = set(campaign_links)
campaign_links = list(set_campaign_links)
print("Anzahl der Campaign links:" + str(len(campaign_links)))
print("Pages: "+str(last_page))
print("Failed: "+str(cum_fails))


Base links: 
['/wg-zimmer-in-Koeln-Holweide.9758772.html', '/wg-zimmer-in-Koeln-Altstadt-Sued.6068399.html', '/wg-zimmer-in-Koeln-Marienburg.3216416.html', '/wg-zimmer-in-Koeln-Kalk.11012234.html', '/wg-zimmer-in-Koeln-Ehrenfeld.11012649.html', '/wg-zimmer-in-Koeln-Kalk.9747118.html', '/wg-zimmer-in-Koeln-Altstadt-Nord.9938808.html', '/wg-zimmer-in-Koeln-Altstadt-Sued.11011997.html', '/wg-zimmer-in-Koeln-Humboldt-Gremberg.11015946.html', '/wg-zimmer-in-Koeln-Loevenich.2085119.html', '/wg-zimmer-in-Koeln-Nippes.5464651.html', '/wg-zimmer-in-Koeln.11008666.html', '/wg-zimmer-in-Koeln-Ehrenfeld.8187268.html', '/wg-zimmer-in-Koeln-Muelheim.11013052.html', '/wg-zimmer-in-Koeln-West.6980083.html', '/wg-zimmer-in-Koeln-Muelheim.10451742.html', '/wg-zimmer-in-Koeln-Brueck.10036288.html', '/wg-zimmer-in-Koeln-Riehl.4764812.html', '/wg-zimmer-in-Koeln-Neustadt-Nord.10729826.html', '/wg-zimmer-in-Koeln-Suelz.11016040.html', '/wg-zimmer-in-Koeln-Suelz.11016040.html', '/wg-zimmer-in-Koeln-Dellbruec

nur verifizierte Unternehmen haben die Klasse "campaign clicks" --> sind auch unten als Werbung aufgeführt

verfifizierte Personen nicht



416 Ergebnisse sind okay. Erwartet hätten wir 420 -> ggf. Werbungen?

In [149]:
# TODO: remove wrong links in the camoaign links (asset_id...)
# TODO: für eine Einzelseite die Attribute bestimmen
